---
**In this project, we will build a sentiment analysis using yelp and distillBERT**

---

# **Imports and IDE Configs**

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertModel
from datasets import Dataset #hugginface datasets

# **Download dataset**

In [ ]:
!git clone https://github.com/KhizarAziz/end-to-end-ML-Pipeline.git
!mv end-to-end-ML-Pipeline/* ./
!rm -rf end-to-end-ML-Pipeline

In [ ]:
!mkdir ~/.kaggle/
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir datasets
!python my_datasets/download_kaggle_dataset.py yelp-dataset/yelp-dataset --path my_datasets/

In [ ]:
!rm -rf datasets/yelp-dataset.zip

# **Load dataset and EDA**

In [ ]:
chunk_size = 1000  # adjust based on available memory
dataset_path = "/content/my_datasets/yelp-dataset/yelp_academic_dataset_review.json"
json_reader = pd.read_json(dataset_path, lines=True, chunksize=chunk_size) # this get all the chunks of json file (as the file is too large, cannot load all at once)
df = next(json_reader) # loading first chunk

In [ ]:
df.to_csv('yelp_academic_dataset_review.csv', index=False)

In [ ]:
pd.read_csv('yelp_academic_dataset_review.csv')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
# Assuming your DataFrame is named df and has a 'stars' column
df['stars'].value_counts().sort_index().plot(kind='bar', title='Star Ratings Distribution')

# **Data Processing**

Data Cleaning


In [ ]:
# Text cleaning function (customize as needed)
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '[URL]', text)
    # Remove Emails
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)
    # Remove new line and line breaks
    text = text.replace('\n', ' ').replace('\r', '').strip()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Optional: Lowercase (Depends on whether your DistilBERT model is cased or not)
    # text = text.lower()

    return text


In [ ]:
# filtering
df.drop(columns=['review_id','user_id','business_id'],inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
# Clean the 'text' column
df['clean_text'] = df['text'].apply(clean_text)

Data Feature Engineering

In [ ]:
# our labels are number of stars which are 1-5 and model expects 0-N (0-4) --->>>> ordinal encoding. So...
# kind of encoding (blkay actually encoding. just coincidentally numbers almost same hn)
df['stars'] = df['stars'] - 1 # feature engineering

Preprocessing

In [ ]:
# validation splitting
train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
# model needs int64 data and this stars originally were float.
train_df['stars'] = train_df['stars'].astype(int)
val_df['stars'] = val_df['stars'].astype(int)

In [ ]:
# preparing huggingface datsets with expected column names (for the tokenizer and model)
train_data = Dataset.from_pandas(train_df[['text', 'stars']].rename(columns={'text': 'text', 'stars': 'label'}))
val_data = Dataset.from_pandas(val_df[['text', 'stars']].rename(columns={'text': 'text', 'stars': 'label'}))

In [ ]:
# dump data into S3
# Using Hugging Face's Datasets
train_data.save_to_disk("./datasets/train_data/")
val_data.save_to_disk("./datasets/val_data/")

# **Model Loading and Training**

In [ ]:
# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [ ]:
from datasets import load_from_disk
train_data = load_from_disk("./datasets/train_data/")
val_data = load_from_disk("./datasets/val_data/")

In [ ]:
# tokenize dataset
train_data_tokenized = train_data.map(tokenize_function, batched=True)
val_data_tokenized = val_data.map(tokenize_function, batched=True)

In [ ]:
# hugging face's this model is in torch, so we need to convert the type to torch dataset.
# Format for pytorch
train_data_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_data_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs'
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(train_data_tokenized['label'].unique()))

In [ ]:
trainer = Trainer( #hugggingface trainer module takes care of parallel trainig.
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=val_data_tokenized
)

In [ ]:
num_labels = len(train_data_tokenized['label'].unique())
model_config_num_labels = model.config.num_labels

print(f"Dataset num_labels: {num_labels}")
print(f"Model config num_labels: {model_config_num_labels}")

In [ ]:
trainer.train()

In [ ]:
#save to s3
model.save_pretrained("./models/")

Script Testing

In [ ]:
!git clone https://github.com/KhizarAziz/end-to-end-ML-Pipeline.git
!mv end-to-end-ML-Pipeline/* ./

In [ ]:
# preprocessing
!python ./preprocessing/preprocess_yelp_review_dataset.py --dataset_path ./datasets/yelp-dataset/yelp_academic_dataset_review.json --chunk_size 1000

In [ ]:
# train
!python ./train.py --train_data_path ./datasets/yelp-dataset/train_set --val_data_path ./datasets/yelp-dataset/train_set

In [ ]:
# p = "/content/datasets/yelp-dataset/yelp_academic_dataset_review.json"
# '/'.join(p.split('/')[:-1])+ '/train_set/'
# train_dumpyard_path = args.dataset_path.split('/')

# **inference**

In [ ]:
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [ ]:
test_text = ["food is awesome!","It was best","I hate it"]
test_encodings = tokenizer(test_text, padding=True, truncation=True, return_tensors='pt')
test_dataset = Dataset.from_dict(test_encodings)
test_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

predictions = trainer.predict(test_dataset)

import torch
import torch.nn.functional as F

logits = torch.tensor(predictions.predictions)
probabilities = F.softmax(logits, dim=0)
print(probabilities)

In [ ]:
print(predictions.predictions[0]),predictions.predictions[1]

In [ ]:
predictions.predictions[2]

In [ ]:
predicted_label_idx = torch.argmax(predictions.predictions[0]).item()